# 灰色模型家族

## 1：灰色预测模型与灰色预测灾变模型（GM（1，1））

灰色预测模型的特点就是：
- 数据量需求少
- 短期预测
- 简单
- 非负数

要求：数据集合必须要达到一定的要求，数据需要有稳定的增长趋势，若剧烈震荡则无法使用
### 检验
所以根据上面的特征，在进行灰色预测之前需要对模型进行‘级比检验’公式如下：
$$ \alpha(x) = \frac{x^{(n)}(k)}{x^{(n)}(k-1)} $$
如果结果在$（e^{-\frac{2}{n+1}},e^{\frac{2}{n+1}}）$之间则数据符合标准





### 2：级比检验

In [14]:
import numpy as np
from typing import Tuple, List, Union

def grade_ratio_test(data: Union[np.ndarray, List],
                     verbose: bool = True) -> Tuple[bool, np.ndarray]:
    """
    数据级比检验

    参数:
    data: 输入数据，一维数组
    verbose: 是否打印详细信息

    返回:
    (是否通过检验, 级比值)

    可容覆盖区间: (e^(-2/(n+1)), e^(2/(n+1)))
    """
    # 转换为numpy数组
    if isinstance(data, list):
        data = np.array(data)

    # 确保是一维数组
    if data.ndim > 1:
        data = data.flatten()

    n = len(data)

    # 计算级比
    grade_ratios = data[:-1] / data[1:]

    # 计算可容覆盖区间
    lower_bound = np.exp(-2 / (n + 1))
    upper_bound = np.exp(2 / (n + 1))

    # 检查级比是否在可容覆盖区间内
    in_interval = (grade_ratios >= lower_bound) & (grade_ratios <= upper_bound)
    all_in_interval = np.all(in_interval)

    if verbose:
        print("=" * 50)
        print("级比检验结果")
        print("=" * 50)
        print(f"原始数据: {data}")
        print(f"数据个数 (n): {n}")
        print(f"可容覆盖区间: ({lower_bound:.4f}, {upper_bound:.4f})")
        print("-" * 50)
        print("各级比检验:")

        for i, (ratio, is_ok) in enumerate(zip(grade_ratios, in_interval)):
            status = "✅ 通过" if is_ok else "❌ 不通过"
            print(f"  λ({i+2}) = {data[i]}/{data[i+1]} = {ratio:.4f} {status}")

        print("-" * 50)
        if all_in_interval:
            print("✅ 检验结果: 所有级比都在可容覆盖区间内，数据适合GM(1,1)建模")
        else:
            print("❌ 检验结果: 存在级比不在可容覆盖区间内，数据不适合GM(1,1)建模")
            print("   建议进行数据平移处理后再尝试建模")
        print("=" * 50)

    return all_in_interval, grade_ratios

def data_transformation(data: np.ndarray, c: float = None) -> np.ndarray:
    """
    数据平移处理

    参数:
    data: 原始数据
    c: 平移常数，如果不指定，自动计算最小值

    返回:
    平移后的数据
    """
    if c is None:
        c = np.abs(np.min(data)) + 1 if np.min(data) <= 0 else 0

    transformed_data = data + c

    print(f"平移处理: 原始数据 + {c} = 新数据")
    print(f"平移后数据: {transformed_data}")

    return transformed_data

# 使用示例
if __name__ == "__main__":
    # 示例1: 通过检验的数据
    print("\n示例1: 通过检验的数据")
    data1 = np.array([889, 2278, 337, 3.390, 679])
    is_ok1, ratios1 = grade_ratio_test(data1)

    if is_ok:
        print("✅ 数据检验通过，可以直接进行GM(1,1)建模")
    else:
        print("❌ 数据检验不通过，建议进行数据平移处理后再建模")
        # 尝试自动平移处理
        c_value = np.abs(np.min(test_data)) + 1 if np.min(test_data) <= 0 else 1
        new_data = data_transformation(test_data, c=c_value)
        is_ok_new, _ = grade_ratio_test(new_data)
        if is_ok_new:
            print("✅ 平移后数据检验通过，可以使用新数据进行GM(1,1)建模")
        else:
            print("❌ 平移后数据仍不满足要求，需要重新选择数据或调整平移参数")


示例1: 通过检验的数据
级比检验结果
原始数据: [ 889.   2278.    337.      3.39  679.  ]
数据个数 (n): 5
可容覆盖区间: (0.7165, 1.3956)
--------------------------------------------------
各级比检验:
  λ(2) = 889.0/2278.0 = 0.3903 ❌ 不通过
  λ(3) = 2278.0/337.0 = 6.7596 ❌ 不通过
  λ(4) = 337.0/3.39 = 99.4100 ❌ 不通过
  λ(5) = 3.39/679.0 = 0.0050 ❌ 不通过
--------------------------------------------------
❌ 检验结果: 存在级比不在可容覆盖区间内，数据不适合GM(1,1)建模
   建议进行数据平移处理后再尝试建模
✅ 数据检验通过，可以直接进行GM(1,1)建模


#### 2.1平移处理
平移处理用于：数据出现负数或者是0值

处理方式：
- 所有数据加上一个常数，这个常数一般为|x_min|+1

### 3：灰色预测模型操作(GM(1,1)，灾变预测和灰色关联度分析)
下面是基本代码

In [11]:
import numpy as np
from typing import Union, List, Tuple, Dict

class GreySystem:
    """灰色系统工具箱"""

    @staticmethod
    def GM11(x0: np.ndarray, predict_steps: int = 1) -> np.ndarray:
        """标准GM(1,1)灰色预测模型"""
        # 确保输入为一维数组
        if x0.ndim > 1:
            x0 = x0.flatten()

        n = len(x0)

        # 一次累加生成
        x1 = np.cumsum(x0)

        # 构造背景值
        z1 = 0.5 * (x1[:-1] + x1[1:])

        # 构建矩阵
        B = np.vstack([-z1, np.ones(n-1)]).T
        Y = x0[1:].reshape(-1, 1)

        # 最小二乘估计
        params = np.linalg.inv(B.T @ B) @ B.T @ Y
        a, b = params[0, 0], params[1, 0]

        # 时间响应式
        def predict(k):
            return (x0[0] - b/a) * np.exp(-a*k) + b/a

        # 生成预测
        x1_pred = np.array([predict(i) for i in range(n + predict_steps)])
        x0_pred = np.concatenate([[x0[0]], np.diff(x1_pred)])

        return x0_pred[-predict_steps:] if predict_steps > 0 else x0_pred

    @staticmethod
    def disaster_forecast(disaster_years: Union[np.ndarray, List], predict_num: int = 1) -> np.ndarray:
        """灾变预测"""
        if isinstance(disaster_years, list):
            disaster_years = np.array(disaster_years)

        if disaster_years.ndim > 1:
            disaster_years = disaster_years.flatten()

        # 转换为时间间隔序列
        intervals = np.diff(disaster_years)

        # 使用GM(1,1)预测未来时间间隔
        pred_intervals = GreySystem.GM11(intervals, predict_num)

        # 计算预测的灾变年份
        disaster_years_pred = []
        last_year = disaster_years[-1]

        for interval in pred_intervals:
            last_year += interval
            disaster_years_pred.append(last_year)

        return np.array(disaster_years_pred)

    @staticmethod
    def grey_incidence_degree(x0: Union[np.ndarray, List],
                               xi: Union[np.ndarray, List, List[List]],
                               rho: float = 0.5) -> Union[float, np.ndarray, Dict]:
        """
        灰色关联度分析

        参数:
        x0: 参考序列
        xi: 比较序列（单个序列或多个序列）
        rho: 分辨系数，默认0.5

        返回:
        单个比较序列时返回关联度值
        多个比较序列时返回关联度字典
        """
        # 转换为numpy数组
        if isinstance(x0, list):
            x0 = np.array(x0)
        if isinstance(xi, list):
            xi = np.array(xi)

        # 确保x0是一维数组
        if x0.ndim > 1:
            x0 = x0.flatten()

        # 处理单序列和多序列的情况
        if xi.ndim == 1:
            # 单序列情况
            if xi.ndim > 1:
                xi = xi.flatten()

            if len(x0) != len(xi):
                raise ValueError("参考序列和比较序列必须具有相同的长度")

            # 无量纲化
            x0_norm = x0 / x0[0]
            xi_norm = xi / xi[0]

            # 计算差值序列
            delta = np.abs(x0_norm - xi_norm)

            # 两级最小差和最大差
            min_min = delta.min()
            max_max = delta.max()

            # 防止除零错误
            if max_max == 0:
                return 1.0

            # 关联系数
            relation_coef = (min_min + rho * max_max) / (delta + rho * max_max)

            # 灰色关联度
            return relation_coef.mean()

        else:
            # 多序列情况
            # xi应该是二维数组，每行是一个比较序列
            n_series, n_points = xi.shape

            if len(x0) != n_points:
                raise ValueError("参考序列和所有比较序列必须具有相同的长度")

            # 1. 无量纲化
            x0_norm = x0 / x0[0]
            xi_norm = xi / xi[:, 0:1]  # 保持维度，每行除以其第一个元素

            # 2. 计算所有比较序列与参考序列的差值序列
            deltas = np.abs(x0_norm - xi_norm)

            # 3. 计算全局最小差和最大差（来自所有比较序列）
            global_min = deltas.min()
            global_max = deltas.max()

            # 4. 计算每个比较序列的关联系数和关联度
            degrees = {}

            for i in range(n_series):
                # 当前序列的差值
                delta_i = deltas[i]

                # 防止除零错误
                if global_max == 0:
                    degree = 1.0
                else:
                    # 关联系数
                    relation_coef_i = (global_min + rho * global_max) / (delta_i + rho * global_max)
                    # 关联度
                    degree = relation_coef_i.mean()

                degrees[f'序列{i+1}'] = degree

            return degrees


# 使用示例
if __name__ == "__main__":
    # 示例1: 标准GM(1,1)预测
    print("=== 示例1: 标准GM(1,1)预测 ===")
    data = np.array([2.874, 999999, 555555, 3.390, 3.679])
    predictions = GreySystem.GM11(data, predict_steps=2)
    print(f"原始数据: {data}")
    print(f"未来2个预测值: {predictions}")
    print()

    # 示例2: 灾变预测
    print("=== 示例2: 灾变预测 ===")
    disaster_data = [2010, 2013, 2017, 2020, 2023]
    disaster_pred = GreySystem.disaster_forecast(disaster_data, 2)
    print(f"历史灾变年份: {disaster_data}")
    print(f"预测灾变年份: {np.round(disaster_pred)}")
    print()

    # 示例3: 单序列灰色关联度分析
    print("=== 示例3: 单序列灰色关联度分析 ===")
    reference = np.array([2, 3, 5, 7, 9])
    compare = np.array([1.8, 3.2, 4.8, 7.1, 8.9])
    degree = GreySystem.grey_incidence_degree(reference, compare)
    print(f"参考序列: {reference}")
    print(f"比较序列: {compare}")
    print(f"灰色关联度: {degree:.4f}")
    print()

    # 示例4: 多序列灰色关联度分析（正确的全局计算方法）
    print("=== 示例4: 多序列灰色关联度分析（全局计算）===")
    reference_seq = np.array([1.0, 2.0, 3.0, 4.0, 5.0]) #目标序列

    # 多个比较序列（需要同维度）
    comparison_seqs = np.array([
        [0.9, 1.8, 2.7, 3.6, 4.5],     # 序列1
        [1, 1.2, 3, 4, 5.5],     # 序列2
        [0.9, 0.6, 2.4, 2, 4.0],     # 序列3
        [1.2, 1.9, 2.9, 4.1, 5.2]      # 序列4
    ])

    print(f"参考序列: {reference_seq}")
    for i, seq in enumerate(comparison_seqs, 1):
        print(f"比较序列{i}: {seq}") # seq是计算完成后的相似系数

    # 使用全局方法计算关联度
    degrees = GreySystem.grey_incidence_degree(reference_seq, comparison_seqs)

    print("\n灰色关联度（全局计算）:")
    for name, degree in degrees.items():
        print(f"{name}: {degree:.4f}")
#### 这里是按照关联度排序
    # 按关联度排序
    sorted_degrees = sorted(degrees.items(), key=lambda x: x[1], reverse=True)

    print("关联度排序（从高到低）:")
    for name, degree in sorted_degrees:
        print(f"{name}: {degree:.4f}")

=== 示例1: 标准GM(1,1)预测 ===
原始数据: [2.87400e+00 9.99999e+05 5.55555e+05 3.39000e+00 3.67900e+00]
未来2个预测值: [24453.58713791  9690.6582451 ]

=== 示例2: 灾变预测 ===
历史灾变年份: [2010, 2013, 2017, 2020, 2023]
预测灾变年份: [2025. 2027.]

=== 示例3: 单序列灰色关联度分析 ===
参考序列: [2 3 5 7 9]
比较序列: [1.8 3.2 4.8 7.1 8.9]
灰色关联度: 0.5365

=== 示例4: 多序列灰色关联度分析（全局计算）===
参考序列: [1. 2. 3. 4. 5.]
比较序列1: [0.9 1.8 2.7 3.6 4.5]
比较序列2: [1.  1.2 3.  4.  5.5]
比较序列3: [0.9 0.6 2.4 2.  4. ]
比较序列4: [1.2 1.9 2.9 4.1 5.2]

灰色关联度（全局计算）:
序列1: 1.0000
序列2: 0.8333
序列3: 0.6152
序列4: 0.6920
关联度排序（从高到低）:
序列1: 1.0000
序列2: 0.8333
序列4: 0.6920
序列3: 0.6152


### 4:精度检验

精度检验是根据GM（1，1）预测函数预测已有的值，然后进行相关的处理其公式如下：
$${\frac{1}{n}}\sum_{n=1}^{n}{\frac{x_n^{(1)}-x_n^{(0)}}{x_n^{(0)}}} $$

